# Aprendamos a usar Julia en Paralllllelo.

Primero que nada, la función nprocs nos dice cuantos cores tenemos. Como estamos empezando el noptebook por default en un solo core, esperamos ver 1.


In [4]:
nprocs()

1

Ahora añadimos otros cores.

In [5]:
addprocs(4)

4-element Array{Any,1}:
 2
 3
 4
 5

In [6]:
nprocs()

5

In [7]:
@parallel for j=2:5
    run(`hostname`)
end

	From worker 4:	mezcalina
	From worker 5:	mezcalina
	From worker 2:	mezcalina
	From worker 3:	mezcalina


La celda anterior corre un comando del sistema en los cuatro cores nombrados

In [8]:
tic();
nheads = @parallel (+) for i=1:100000000
    rand()
end;
s=toc();
println("Suma aleatorea en paralelo: $nheads in $s seconds")

elapsed time: 0.60485369 seconds
Suma aleatorea en paralelo: 5.000155206076304e7 in 0.60485369 seconds


In [9]:
tic();
a=0
for i=1:100000000
    a+=rand()
end;
toc();
println(a)

elapsed time: 5.353554791 seconds
4.999843790615058e7


In [10]:
referencia= @spawn rand(8,8)

RemoteRef(4,1,24)

In [11]:
A=fetch(referencia);

In [12]:
@everywhere function probando(x::Real)
    return exp(-x^2)
end


In [13]:
probando(3)-probando(-3)

0.0

Esto NO se puede hacer. La funcion tiene que ser reconocida por el proceso:
```
@everywhere lulu=probando()
```
en cambio parece ser que esto si se puede hacer

In [14]:
@everywhere lurdes()=randn()

Entonces `lulu` ya podria ser llamada

In [15]:
remotecall_fetch(2, ()->lulu) #¿Como paserle argumentos a lulu?

exception on 2: ERROR: lulu not defined
 in anonymous at In[15]:1
 in anonymous at multi.jl:855
 in run_work_thunk at multi.jl:621
 in anonymous at task.jl:855


UndefVarError(:lulu)

In [16]:
remotecall_fetch(3, probando,2)

0.01831563888873418

In [17]:
?remotecall_fetch

INFO: Loading help data...


Base.remotecall_fetch(id, func, args...)

   Perform "fetch(remotecall(...))" in one message.


In [18]:
probandosumagaussiana= @parallel (+) for j=1:10000000
    probando(randn())
end

5.775406851804929e6

In [19]:
M={rand(2000,2000) for i=1:4};

Eso es un array de matrices de 1000 por 1000.

In [20]:
@time tururu=pmap(eig,M);

elapsed time: 15.097643122 seconds (432300840 bytes allocated, 0.34% gc time)


Esa madre mapeo usando los procesadores disponinble la funcion `eig` sobre el conjunto $M$.